In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np

from prophet import Prophet
from datetime import timedelta

from sklearn.ensemble import RandomForestRegressor

In [3]:
from xgboost import XGBRegressor

In [9]:
# history = pd.read_csv("/masters_diploma/raw_data/open-meteo-50.44N30.48E188m.csv", header=2).dropna()
history = pd.read_csv("/masters_diploma/processed_data/history_weather.csv")

history = history.rename(columns={history.columns[0]: "date"})
history["date"] = pd.to_datetime(history["date"], errors="coerce")
history = history.set_index("date").sort_index()

In [10]:
history

,temperature,relative_humidity,dew_point,precipitation,rain,snowfall,weather_code,pressure_msl,surface_pressure,cloud_cover,...,sunshine_duration_max_3_years,sunshine_duration_min_3_years,sunshine_duration_mean_3_years,sunshine_duration_max_4_years,sunshine_duration_min_4_years,sunshine_duration_mean_4_years,sunshine_duration_lag_24,sunshine_duration_lag_48,sunshine_duration_lag_72,sunshine_duration_lag_168
date,,,,,,,,,,,,,,,,,,,,,
2015-01-01 00:00:00,-5.9,56.0,-13.3,0.0,0.0,0.07,71.0,1026.1,1001.8,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-01-01 01:00:00,-5.5,59.0,-12.4,0.0,0.0,0.00,3.0,1025.1,1000.8,97.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-01-01 02:00:00,-5.2,63.0,-11.2,0.0,0.0,0.00,3.0,1024.7,1000.5,100.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-01-01 03:00:00,-4.9,68.0,-9.9,0.0,0.0,0.00,3.0,1024.3,1000.1,99.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2015-01-01 04:00:00,-4.6,73.0,-8.7,0.0,0.0,0.07,71.0,1024.1,1000.0,98.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-09-25 19:00:00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-09-25 20:00:00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2025-09-25 21:00:00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
rf_hp = {'max_depth': 7, 'n_estimators': 500, 'n_jobs': -1, 'random_state': 2}
xgb_hp = {
        'n_estimators':1000,
        'n_jobs':-1,
        'max_depth':8,
        'eta':0.3,
        'booster':'gbtree',
        'objective':'reg:squarederror',
        'eval_metric':'rmse',
        'subsample':1,
        'colsample_bytree':1,
        'min_child_weight':1,
        'random_state':2,
}


# train_feature = ['month', 'year_day', 'is_day', 'cloud_cover_mean_7_days', 'cloud_cover_max_3_years', 'sunshine_duration_min_7_days', 
#                  'temperature_mean_7_days', 'temperature_lag_168', 'temperature_mean_3_years']

train_feature = ['month', 'temperature_lag_168', 'temperature_mean_3_years']

train_start = '2024-01-01 00:00:00'
train_end = '2025-09-15 23:00:00'
test_start = '2025-09-16 00:00:00'
test_end = '2025-09-18 23:00:00'

days_forecast = 7

In [54]:
model_rf = RandomForestRegressor(**rf_hp)
model_xgb = XGBRegressor(**xgb_hp)

In [55]:
X_train = history.loc[train_start:train_end, train_feature]
X_test = history.loc[test_start:test_end, train_feature]

y_train = history.loc[train_start:train_end, 'temperature']
y_test = history.loc[test_start:test_end, ['temperature']]

In [56]:
y_pred = model_rf.fit(X_train, y_train).predict(X_test)

In [57]:
y_test['pred'] = y_pred

In [50]:
y_test

,temperature,pred
date,,
2025-09-16 00:00:00,14.4,16.643849
2025-09-16 01:00:00,14.5,15.597643
2025-09-16 02:00:00,14.3,15.578128
2025-09-16 03:00:00,14.0,15.442785
2025-09-16 04:00:00,13.5,15.398214
...,...,...
2025-09-18 19:00:00,13.7,16.070799
2025-09-18 20:00:00,13.1,15.988380
2025-09-18 21:00:00,12.7,15.929126


In [58]:
err = (y_test['pred']/y_test['temperature'] - 1) * 100
err.abs().mean()

27.774049602242872

## feature engineering

In [ ]:
# sunrise_sunset = pd.read_csv("./sunset_sunrise.csv").dropna()

# for col in sunrise_sunset.columns:
#     if col != "time":
#         sunrise_sunset[col] = pd.to_datetime(sunrise_sunset[col]) + timedelta(hours=3)
# sunrise_sunset = sunrise_sunset.set_index("time")
# sunrise_sunset.columns = [k.split()[0] for k in sunrise_sunset.columns]


# time_start = [n.hour if n.minute < 30 else n.hour+1 for n in sunrise_sunset['sunrise']]
# time_end = [n.hour if n.minute < 30 else n.hour+1 for n in sunrise_sunset['sunset']]

# day_length = list(zip(time_start, time_end))
# sunrise_sunset['day_night'] = day_length

# sunrise_sunset

In [ ]:
# history['hour'] = history.index.hour
# history['day'] = history.index.day
# history['month'] = history.index.month
# history['week_day'] = history.index.day_of_week
# history['year_day'] = history.index.day_of_year
# history['year'] = history.index.year

In [ ]:
# for row in history.index:
#     day_hours = sunrise_sunset.loc[row.strftime("%Y-%m-%d"), 'day_night']
#     history.loc[row, 'day_night'] = 1 if day_hours[0] <= row.hour <= day_hours[1] else 0

In [ ]:
# _new_cols = (
#     history.groupby([history.index.month, history.index.day, history.index.hour])[['temperature_2m', 'relative_humidity_2m', 'precipitation']]
#     .rolling(window=3, closed='left')
#     .agg(['mean', 'min', 'max'])
#     .reset_index(level=[0, 1, 2], drop=True)
#     .abs()
# )

# _new_cols.columns = [f'temp_max_3_days', f'temp_min_3_days', f'temp_mean_3_days', 
#                      f'hum_max_3_days', f'hum_min_3_days', f'hum_mean_3_days',
#                      f'prec_max_3_days', f'prec_min_3_days', f'prec_mean_3_days']
# day_mean = _new_cols.reset_index()[['time', f'temp_max_3_days', f'temp_min_3_days', f'temp_mean_3_days', 
#                                     f'hum_max_3_days', f'hum_min_3_days', f'hum_mean_3_days',
#                                     f'prec_max_3_days', f'prec_min_3_days', f'prec_mean_3_days']]

# history = history.merge(day_mean, on='time').set_index("time").fillna(0)

# _new_cols = (
#     history.groupby([history.index.day_of_year])[['temperature_2m', 'relative_humidity_2m', 'precipitation']]
#     .rolling(window=3, closed='left')
#     .agg(['mean', 'min', 'max'])
#     .reset_index(level=[0], drop=True)
#     .abs()
# )

# _new_cols.columns = [f'temp_max_3_years', f'temp_min_3_years', f'temp_mean_3_years', 
#                      f'hum_max_3_years', f'hum_min_3_years', f'hum_mean_3_years',
#                      f'prec_max_3_years', f'prec_min_3_years', f'prec_mean_3_years']
# day_mean = _new_cols.reset_index()[['time', f'temp_max_3_years', f'temp_min_3_years', f'temp_mean_3_years', 
#                                     f'hum_max_3_years', f'hum_min_3_years', f'hum_mean_3_years',
#                                     f'prec_max_3_years', f'prec_min_3_years', f'prec_mean_3_years']]

# history = history.merge(day_mean, on='time').set_index("time").fillna(0)

# _new_cols = (
#     history.groupby([history.index.hour])[['temperature_2m', 'relative_humidity_2m', 'precipitation']]
#     .rolling(window=3, closed='left')
#     .agg(['mean', 'min', 'max'])
#     .reset_index(level=[0], drop=True)
#     .abs()
# )

# _new_cols.columns = [f'temp_max_3_hours', f'temp_min_3_hours', f'temp_mean_3_hours', 
#                      f'hum_max_3_hours', f'hum_min_3_hours', f'hum_mean_3_hours',
#                      f'prec_max_3_hours', f'prec_min_3_hours', f'prec_mean_3_hours']
# day_mean = _new_cols.reset_index()[['time', f'temp_max_3_hours', f'temp_min_3_hours', f'temp_mean_3_hours', 
#                                     f'hum_max_3_hours', f'hum_min_3_hours', f'hum_mean_3_hours',
#                                     f'prec_max_3_hours', f'prec_min_3_hours', f'prec_mean_3_hours']]

# history = history.merge(day_mean, on='time').set_index("time").fillna(0)

In [ ]:
# history.columns

In [ ]:
err

In [ ]:
y_test